# The Rank/Atop operator: `⍤`

> An algorithm must be seen to be believed.
\--_Don Knuth_

The somewhat startled-looking rank operator, `⍤`, has a reputation for being one of the harder ones to grasp. It sits on the 'J' key, a handy mnemonic, as it was an invention borrowed from the [J-language](https://www.jsoftware.com/#/).

If `⍤` is given a right operand _function_ it becomes `atop`. We'll talk about that after we've covered the `rank` version.

Other resources on rank and atop:
* [APL Wiki](https://aplwiki.com/wiki/Rank_(operator))
* Dyalog docs [Rank](https://help.dyalog.com/latest/Content/Language/Primitive%20Operators/Rank.htm), [Atop](https://help.dyalog.com/latest/index.htm#Language/Primitive%20Operators/Atop.htm)
* Webinars: [Rank Operator](https://dyalog.tv/Webinar/?v=IBct81IopRk), [Advanced Rank](https://dyalog.tv/Webinar/?v=5wW76XX0kqk), [Rank and Dyadic Transpose](https://dyalog.tv/Webinar/?v=zBqdeDJPPRc)
* APL Orchard cultivations: [Basic Rank](https://chat.stackexchange.com/rooms/52405/conversation/lesson-32-basic-use-of-), [Advanced Rank](https://chat.stackexchange.com/rooms/52405/conversation/lesson-33-advanced-use-of-), [Atop and Over](https://chat.stackexchange.com/rooms/52405/conversation/lesson-47-atop--and-over-)

In [1]:
⎕IO ← 0
{}⎕SE.UCMD'box on -s=min -t=tree -f=on'
{}⎕SE.UCMD'rows on'

done

Rebuilding user command cache... done

In essence, the simple use of rank drills into the argument array and applies its operand to sub cells of a specified rank only. 

Consider the following rank-3 array:

In [4]:
⊢m ← 3 3 3⍴27?27
≢⍴m

┌┌→───────┐
↓↓16 22 0│
││15 9 6│
││11 8 21│
││ │
││ 1 20 17│
││ 3 18 23│
││ 5 7 14│
││ │
││13 24 25│
││12 10 2│
││26 4 19│
└└~───────┘

3

Let's say we want to drop the first row of each layer of this array. In this case, we could use a bracket-axis specification to drop:

In [5]:
1↓[1]m ⍝ Apply drop ↓ to axis 1, i.e. the y-axis, with ⎕IO←0

┌┌→───────┐
↓↓15 9 6│
││11 8 21│
││ │
││ 3 18 23│
││ 5 7 14│
││ │
││12 10 2│
││26 4 19│
└└~───────┘

However, this is now `⎕IO` dependent, and the bracket-axis spec has some other undesirable properties, including the fact that it cannot be applied to user-defined functions. 

The rank operator allows us to solve this in a different way:

In [6]:
1↓⍤2⊢m

┌┌→───────┐
↓↓15 9 6│
││11 8 21│
││ │
││ 3 18 23│
││ 5 7 14│
││ │
││12 10 2│
││26 4 19│
└└~───────┘

You can think of rank in this case being closer to the problem statement: drop the first major cell of each _sub cell_ that has a rank one less than the argument. Or: in our 3-D array, apply the drop to each of its constituent 2-D arrays.

Note that rank doesn't equal _depth_, and this is part of the reason why the rank operator can be confusing. So given a complex vector that contains a variety of scalars and higher-rank arrays, you can't use the rank operator to selectively apply a function to, say, just the vectors.

In [14]:
⊢V ← (1 2 3)(2 2⍴1 1 1 1)(1)

┌→────────────────┐
│ ┌→────┐ ┌→──┐ │
│ │1 2 3│ ↓1 1│ 1 │
│ └~────┘ │1 1│ │
│ └~──┘ │
└∊────────────────┘

In [13]:
≢⍤1⊢V ⍝ NOTE: this is wrong!

3

In this case, `V` is of course already a rank-1 array (i.e. a vector). But wait, there is more confusion to be had! The right operand doesn't even have to be a scalar at all.

We already saw that if the right operand of `⍤` is a scalar, we are specifying which rank sub cells we want a function to apply to. For dyadic usage, we instead specify which sub cells of the left argument should be paired up with which sub cells of the right argument. It takes a while to wrap your head around this.

Here's an example:

In [15]:
1 2 3,⍤0 1 ⊢ 'hello'

┌→──────┐
↓1 hello│
│2 hello│
│3 hello│
└+──────┘

This says that the _left_ argument of the derived function should be paired at rank 0 (that is each element) with the _right_ argument at rank 1 (that is the whole vector in this case). If this feels reminiscent of an outer product, you're not wrong.

In other words, rank always pairs up its arguments. We can examine which elements would be paired up by using the following incantation in place of our function:

In [18]:
rankpairs ← ⊂⍤,⍥⊂

if you know what that means, you probably don't need to read this tutorial. If not, don't worry -- treat it as a debug statement:

In [19]:

1 2 3 rankpairs⍤0 1 ⊢ 'hello'

┌→──────────────────────────────────────────┐
│ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ │
│ │ ┌→────┐ │ │ ┌→────┐ │ │ ┌→────┐ │ │
│ │ 1 │hello│ │ │ 2 │hello│ │ │ 3 │hello│ │ │
│ │ └─────┘ │ │ └─────┘ │ │ └─────┘ │ │
│ └∊──────────┘ └∊──────────┘ └∊──────────┘ │
└∊──────────────────────────────────────────┘

In the section on array indexing earlier, we introduced the concept of _Sane Indexing_ without any further explanation on how it was working. Here it is again:

In [20]:
I←⌷⍨∘⊃⍨⍤0 99 ⍝ Sane indexing, a.k.a select

Now that we understand a bit about rank, we can at least make a start in taking that apart. So we have a rank 0 99 to the right, stating that we should combine the left argument at rank 0 (scalars) with the right argument at "rank 99". Rank 99 means "full rank", as Dyalog has a max rank of 99. So we should combine "each scalar" to the left with "the whole thing, at whatever rank it happens to be" to the right.

The actual indexing is done by the leading squad (`⌷`). Let's de-selfie the expression and make it a dfn. We can have the interpreter help us out a bit to see how that expression actually parses:

In [23]:
⌷⍨∘⊃⍨⍤0 99

⍤ 
 ┌┴┐ 
 ⍨ 0 99
 ┌─┘ 
 ∘ 
 ┌┴┐ 
 ⍨ ⊃ 
┌─┘ 
⌷

If we separate out the rank expression, after a bit of head-scratching we arrive at

In [39]:
sane←{(⊃⍺)⌷⍵}

In [40]:
3 sane 'abcdefg'

d
-

In [41]:
1 2 3 sane⍤0 99⊢'abcdefg'

┌→──┐
│bcd│
└───┘

Hopefully, we can now see a bit clearer how it works -- we simply pass the disclosed left argument vector to squad, and rank does the appropriate combination of left and right: each left element, to the whole (full-rank) of the right.

Here's another example. Given two integer arrays of the same shape, can we replicate (or compress) the rows of one from the other? We might be excused if we think that we can replicate-reduce, but this doesn't work:

In [5]:
⊢A ← 3 4⍴2 1 1 1 1 2 1 1 1 1 2 1
⊢B ← 3 4⍴1 2 3 4 1 2 3 4 1 2 3 4

2 1 1 1
1 2 1 1
1 1 2 1

1 2 3 4
1 2 3 4
1 2 3 4

(note: all expressions in the next cell will generate syntax or rank errors)

In [8]:
A∘//B     ⍝ Nope... ERROR
{A/⍵}/B   ⍝ Nope... ERROR
A∘{⍺/⍵}/B ⍝ Nope... ERROR

SYNTAX ERROR: The function does not take a left argument
      A∘//B   ⍝ Nope ERROR
        ∧
RANK ERROR
      {A/⍵}/B ⍝ Nope... ERROR
        ∧
SYNTAX ERROR: The function does not take a left argument
      A∘{⍺/⍵}/B
            ∧


Instead, we need to reach for rank:

In [9]:
A/⍤1⊢B

1 1 2 3 4
1 2 2 3 4
1 2 3 3 4

If we spell it out in plain words, perhaps the use of rank suggests itself: combine each major cell on the left with the corresponding major cell on the right, and apply the dyadic function "replicate". Every time you think "combine each major cell on the left...", you should think "rank".

As we've seen elsewhere, APL has a somewhat unholy mix of leading and trailing axis operators, mainly for historical reasons. Rank allows us to stick to the leading axis versions [[Kromberg and Hui](https://dl.acm.org/doi/pdf/10.1145/3386319)]:

|Leading axis|Trailling axis|Operation|
|:-----------|:-------------|:--------|
|(⊖⍤1)⍵      |⌽⍵           |reverse  |
|⍺(⊖⍤1)⍵     |⍺⌽⍵           |rotate (scalar ⍺)|
|(f⌿⍤1)⍵      |f/⍵           | reduce  |
|(f⍀⍤1)⍵      |f\⍵           | scan |


## Rank drills 

In his paper [APL Excercises](https://www.jsoftware.com/papers/APL_exercises/) Roger Hui sets out the following drills for rank. See if you can predict the results before revealing the answers:

In [4]:
,t←2 3 4⍴⍳24

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24

In [10]:
,⍤3⊢t
]DISPLAY ,⍤2⊢t
]DISPLAY ,⍤1⊢t

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24

┌→──────────────────────────────────┐
↓ 1 2 3 4 5 6 7 8 9 10 11 12│
│13 14 15 16 17 18 19 20 21 22 23 24│
└~──────────────────────────────────┘

┌┌→──────────┐
↓↓ 1 2 3 4│
││ 5 6 7 8│
││ 9 10 11 12│
││ │
││13 14 15 16│
││17 18 19 20│
││21 22 23 24│
└└~──────────┘

In [6]:
⊂⍤3⊢t
⊂⍤2⊢t
⊂⍤1⊢t

┌───────────┐
│ 1 2 3 4│
│ 5 6 7 8│
│ 9 10 11 12│
│ │
│13 14 15 16│
│17 18 19 20│
│21 22 23 24│
└───────────┘

┌──────────┬───────────┐
│1 2 3 4│13 14 15 16│
│5 6 7 8│17 18 19 20│
│9 10 11 12│21 22 23 24│
└──────────┴───────────┘

┌───────────┬───────────┬───────────┐
│1 2 3 4 │5 6 7 8 │9 10 11 12 │
├───────────┼───────────┼───────────┤
│13 14 15 16│17 18 19 20│21 22 23 24│
└───────────┴───────────┴───────────┘

In [11]:
↑ ,¨ ⊂⍤3⊢t
]DISPLAY ↑ ,¨ ⊂⍤2⊢t
]DISPLAY ↑ ,¨ ⊂⍤1⊢t

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24

┌→──────────────────────────────────┐
↓ 1 2 3 4 5 6 7 8 9 10 11 12│
│13 14 15 16 17 18 19 20 21 22 23 24│
└~──────────────────────────────────┘

┌┌→──────────┐
↓↓ 1 2 3 4│
││ 5 6 7 8│
││ 9 10 11 12│
││ │
││13 14 15 16│
││17 18 19 20│
││21 22 23 24│
└└~──────────┘

In [8]:
]DISPLAY +⌿⍤3⊢t
]DISPLAY +⌿⍤2⊢t
]DISPLAY +⌿⍤1⊢t

14 16 18 20
22 24 26 28
30 32 34 36

15 18 21 24
51 54 57 60

10 26 42
58 74 90

In [12]:
]DISPLAY ⊖⍤3⊢t
]DISPLAY ⊖⍤2⊢t
]DISPLAY ⊖⍤1⊢t

┌┌→──────────┐
↓↓13 14 15 16│
││17 18 19 20│
││21 22 23 24│
││ │
││ 1 2 3 4│
││ 5 6 7 8│
││ 9 10 11 12│
└└~──────────┘

┌┌→──────────┐
↓↓ 9 10 11 12│
││ 5 6 7 8│
││ 1 2 3 4│
││ │
││21 22 23 24│
││17 18 19 20│
││13 14 15 16│
└└~──────────┘

┌┌→──────────┐
↓↓ 4 3 2 1│
││ 8 7 6 5│
││12 11 10 9│
││ │
││16 15 14 13│
││20 19 18 17│
││24 23 22 21│
└└~──────────┘

If you got all those right, well done -- Roger's paper has got many more for you to pit your wits against.

## Atop

As we mentioned above, from version 18 of Dyalog, `⍤` can also be `atop` if given a right operand function instead of array. We'll see more of different kinds of `atop` later, but for now it's a functional composition rule, stating that (for the dyadic case),
```apl
X f⍤g Y → f X g Y
```

It is similar enough to _jot_, `∘`, to be confusing. Whilst monadic `∘` is an atop, dyadic `∘` is a _beside_. Behold:
```apl
f∘g X → f(gX) → fgX
f⍤g X → f(gX) → fgX
X f∘g Y → X f g Y
X f⍤g Y → f X g Y
```

As the monadic atop is equivalent for `∘` and `⍤`, it's worth using `⍤` for both monadic and dyadic atops for consistency.